In [1]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

/opt/conda/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [2]:
# for plotting
import matplotlib.pyplot as plt
# for matrix and linear algebra
import numpy as np
# preinstalled dataset
from qiskit.ml.datasets import ad_hoc_data
# basic simulator
from qiskit import BasicAer
# import Quantm Instance to run the experiment
from qiskit.aqua import QuantumInstance
# import quantum feature map
from qiskit.circuit.library import ZZFeatureMap
# import Quantum Support vector Machine model
from qiskit.aqua.algorithms import QSVM
# import several useful unitilies
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
#
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
print("Necessary libraries and packages are imported successfully!")

Necessary libraries and packages are imported successfully!


In [4]:
random_seed = 42
# set shots/ number of iteration
shot = 1000

In [6]:
url = 'https://raw.githubusercontent.com/denniszhao97/MSiA_Hackathon/main/input_data.csv'
data = pd.read_csv(url)

In [7]:
data.loc[data["Median Income"] == "2,500-", "Median Income"] = "2500"
data.loc[data["Median Income"] == "250,000+", "Median Income"] = "250000"
data["Median Income"] = data["Median Income"].astype(float)
data.head()

,Median Income,poverty_rate,percent_completed_hs,share_white,share_black,share_native_american,share_asian,share_hispanic,police_killing
0,11207.0,78.8,21.2,67.2,30.2,0.0,0.0,1.6,0.0
1,25615.0,29.1,69.1,54.4,41.4,0.1,1.0,3.1,1.0
2,42575.0,25.5,78.9,52.3,44.9,0.5,0.3,2.3,0.0
3,37083.0,30.7,81.4,99.1,0.1,0.0,0.1,0.4,0.0
4,21667.0,42.0,68.6,13.2,86.5,0.0,0.0,0.3,0.0


In [13]:
scaler = StandardScaler()
train_input = {}
scaler.fit(X_train)
value_0 = scaler.transform(X_train_resample)
value_1 = scaler.transform(X_train_keep)

train_input['0'] = value_0[:30,:5]
train_input['1'] = value_1[:30,:5]

test_input = {}
my_list = y_test[y_test == 0].index.to_list()
filter_0 = X_test[X_test.index.isin(my_list)]
scaler.fit(X_test)
value_0 = scaler.transform(filter_0)

my_list = y_test[y_test== 1].index.to_list()
filter_1 = X_test[X_test.index.isin(my_list)]
value_1 = scaler.transform(filter_1)

test_input['0'] = value_0[:30,:5]
test_input['1'] = value_1[:30,:5]

In [101]:
url = 'https://raw.githubusercontent.com/denniszhao97/MSiA_Hackathon/main/input_data.csv'
data = pd.read_csv(url,index_col=0)

In [30]:
data.head()

,Median Income,poverty_rate,percent_completed_hs,share_white,share_black,share_native_american,share_asian,share_hispanic,police_killing
0,11207.0,78.8,21.2,67.2,30.2,0.0,0.0,1.6,0.0
1,25615.0,29.1,69.1,54.4,41.4,0.1,1.0,3.1,1.0
2,42575.0,25.5,78.9,52.3,44.9,0.5,0.3,2.3,0.0
3,37083.0,30.7,81.4,99.1,0.1,0.0,0.1,0.4,0.0
4,21667.0,42.0,68.6,13.2,86.5,0.0,0.0,0.3,0.0


In [103]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,:-1], data['police_killing'], test_size=0.3, random_state=1)
feature_dimension = len(X_train.columns)

In [16]:
train_input = {}
scaler = StandardScaler()
scaler.fit(X_train)
value_0 = scaler.transform(X_train_resample)
value_1 = scaler.transform(X_train_keep)

train_input['0'] = value_0[:20,:5]
train_input['1'] = value_1[:20,:5]

test_input = {}
my_list = y_test[y_test == 0].index.to_list()
filter_0 = X_test[X_test.index.isin(my_list)]
scaler.fit(X_test)
value_0 = scaler.transform(filter_0)

my_list = y_test[y_test== 1].index.to_list()
filter_1 = X_test[X_test.index.isin(my_list)]
value_1 = scaler.transform(filter_1)

test_input['0'] = value_0[:38,:5]
test_input['1'] = value_1[:2,:5]

In [17]:
# use the qasm simulator
backend = BasicAer.get_backend('qasm_simulator')
feature_map = ZZFeatureMap(5, reps=2,entanglement='linear')
qsvm = QSVM(feature_map, train_input, test_input, np.concatenate([test_input['0'],test_input['1']]))

# set the random seed for this qsvm
qsvm.random_seed = random_seed
# setup the quantum instance to run the experiment
quantum_instance = QuantumInstance(backend, shots=shot, seed_simulator=random_seed, seed_transpiler=random_seed)

In [18]:
# get the result from the above designed experiment on the quantum_instance
result = qsvm.run(quantum_instance)

In [ ]:
# set the random seed for this qsvm
qsvm.random_seed = random_seed
# setup the quantum instance to run the experiment
quantum_instance = QuantumInstance(backend, shots=shot, seed_simulator=random_seed, seed_transpiler=random_seed)
# get the result from the above designed experiment on the quantum_instance
result = qsvm.run(quantum_instance)

In [20]:
y_pred = result['predicted_labels']

In [27]:
Y_test = [0 for i in range(38)]
Y_test.append(1)
Y_test.append(1)

In [25]:
conf_mat = confusion_matrix(Y_test, temp)

In [26]:
conf_mat

array([[19, 19],
       [ 0,  2]])